In [1]:
from utils import *
from calc_config import *

In [2]:
abs_main, nucleus, pathfilename = pathfilename_gen(pcname,input_txt)
os.chdir(abs_main)

## Reading the txt file and convert to a list object
text_whole = []
with open (pathfilename["source_text"], 'rt') as myfile:    
    for line in myfile:
        line = line.rstrip('\n')
        text_whole.append(line)

# Parameters extractions

In [3]:
## Extraction of num_particles data; Create a new class of object called num_orbitals
## Below the number of particles and orbitals all refer to the valence"s
## Valence!! VALENCE!!

num_particles = [None,None]
num_orbitals = [None,None]
num_spin_orbitals = None

## Extract particles number information
pattern_energy_start = "-------------------- Lower bound for Pairing Valence Space -------------------------"
pattern_energy_end   = "-------------------- Upper bound for Pairing Valence Space -------------------------"
energy_block_id = identify_line(text_whole, pattern_energy_start,pattern_energy_end);
energy_block_id.sort()

energy_neut_list = text_whole[energy_block_id[0]+1: energy_block_id[1]]
energy_prot_list = text_whole[energy_block_id[2]+1: energy_block_id[3]]
energy_neut_list = remove_line(energy_neut_list,"Fermi")
energy_prot_list = remove_line(energy_prot_list,"Fermi")

num_neut_list = []; num_prot_list = []
for line in energy_neut_list:
    num_neut_list.append(int(line[53:55]))
for line in energy_prot_list:
    num_prot_list.append(int(line[53:54]))
    
### Summary of this Block
num_particles[0] = sum(num_neut_list); num_particles[1] = sum(num_prot_list); num_particles = tuple(num_particles)
num_neut_orbitals = len(energy_neut_list); num_prot_orbitals = len(energy_prot_list)
num_orbitals[0] = num_neut_orbitals; num_orbitals[1] = num_prot_orbitals; num_orbitals = tuple(num_orbitals)
num_spin_orbitals = sum(num_orbitals)
### SUMMARY END ###

### Getting the hf_lvl index for the lower bound hf states
n_start = int(energy_neut_list[0][7:10].strip())
p_start = int(energy_prot_list[0][7:10].strip())

# Twobody Matrix Terms source

In [4]:
## Extract twobody matrix elements information
extracted_twobody = []
pattern_hf = re.compile("HF INDEX AND ISOSPIN")
for line in text_whole:
    if pattern_hf.search(line) != None:      # If a match is found
        extracted_twobody.append((line.rstrip('\n')).lstrip('HF INDEX AND ISOSPIN'))
### Store the information extracted into a dataframe; and output them as csv for later viewing/assessment
obs_twobody_df = pd.DataFrame(columns = ["q_i1","q_i2","q_f1","q_f2","V_ffii","hf_lvl_i1","iso_i1","hf_lvl_i2","iso_i2","hf_lvl_f1","iso_f1","hf_lvl_f2","iso_f2"])
for counter, extracted_twobody_row in enumerate(extracted_twobody):
    row_split = extracted_twobody_row.split()
    dummy_q_i1 = np.nan; dummy_q_i2 = np.nan; dummy_q_f2 = np.nan; dummy_q_f2 = np.nan; #q is qubit; i1,i2,f1,and f2  qubit indeces
    dummy_V_ffii = float(row_split[8])
    dummy_hfl_i1 = int(row_split[0]); dummy_iso_i1 = int(row_split[1])
    dummy_hfl_i2 = int(row_split[2]); dummy_iso_i2 = int(row_split[3])
    dummy_hfl_f1 = int(row_split[4]); dummy_iso_f1 = int(row_split[5])
    dummy_hfl_f2 = int(row_split[6]); dummy_iso_f2 = int(row_split[7])

    appending_row = [
        dummy_q_i1,dummy_q_i2,dummy_q_f2,dummy_q_f2,dummy_V_ffii,
        dummy_hfl_i1,dummy_iso_i1,
        dummy_hfl_i2,dummy_iso_i2,
        dummy_hfl_f1,dummy_iso_f1,
        dummy_hfl_f2,dummy_iso_f2]
    obs_twobody_df.loc[counter] = appending_row
obs_twobody_source_df_output = obs_twobody_df.copy()

### Define the qubit indeces
#### Start #### create to make the code cleaner, will be looped in the defining values for qubit indeces
suffix_list = ["i1","i2","f1","f2"]; iso_list=[]; q_list=[]; hf_lvl_list=[]
for suffix in suffix_list:
    iso_list.append("iso_"+suffix); q_list.append("q_"+suffix); hf_lvl_list.append("hf_lvl_"+suffix)
#### End ####
for index, row in obs_twobody_source_df_output.iterrows():
    for (iso_dum,q_dum,hf_lvl_dum) in zip(iso_list, q_list, hf_lvl_list):
        if row[iso_dum] == 1:         ## iso = 1 represents neutron; 2 represents proton
            row[q_dum] = row[hf_lvl_dum] - n_start
        elif row[iso_dum] == 2:
            row[q_dum] = row[hf_lvl_dum] - p_start + num_neut_orbitals

### To csv
obs_twobody_source_df_output.to_csv(pathfilename["output_2B-source_csv"])

# Onebody Matrix Terms source

In [5]:
# Extract onebody information
## Store the onebody terms into panda dataframes and export as csv
obs_onebody_df = pd.DataFrame(columns = ["q_i","spin","epsilon","hf_lvl","iso"])

for counter, line in enumerate(energy_neut_list+energy_prot_list):
    dummy_spin = None
    dummy_iso = None
    dummy_df_row = []
    line = line.replace(')',' ')
    line = remove_character(line,'()')
    split_list = line.split()
    if int(split_list[6]) == 1:
        dummy_iso = 1
    elif int(split_list[6]) == -1:
        dummy_iso = 2
    else: 
        pass

    if  float(Fraction(split_list[2])) < 0:
        dummy_spin = -1
    elif float(Fraction(split_list[2])) > 0:
        dummy_spin = 1
    else:
        pass
    dummy_hf_lvl = int(split_list[1])
    dummy_epsilon = float(split_list[4])
    dummy_df_row = [np.nan,dummy_spin,dummy_epsilon,dummy_hf_lvl,dummy_iso]
    obs_onebody_df.loc[counter] = dummy_df_row
obs_onebody_source_df_output = obs_onebody_df.copy() ## clone the df so that the below codes doesnt mess things up(action is debatable, but i dont care)

### Define the qubit indeces for obs_onebody_df (conver hf_lvl to q_i; different encoding/indexing)
for index, row in obs_onebody_source_df_output.iterrows():
    if row["iso"] == 1:
        row["q_i"] = row["hf_lvl"] - n_start
    elif row["iso"] == 2:
        row["q_i"] = row["hf_lvl"] - p_start + num_orbitals[0]

### To csv
obs_onebody_source_df_output.to_csv(pathfilename['output_1B-source_csv'])
obs_onebody_df = obs_onebody_source_df_output # Change the df back to the changed values

# Filter out unwanted interactions from obs_twobody_df
# Create obs_twobody_H_input_df_output (input for VQE)

In [6]:
##### obs_twobody_df will always be the name of being_worked_df of two
obs_twobody_df = obs_twobody_source_df_output

from itertools import combinations
## Create interaction of Interest aka excitation_list
### Allowed pair combinations
# non_repeat_list = list(combinations(range(0,num_spin_orbitals),2))    ## List pair combination that doesn't repeat same orbitals (particle doesnt occupy same orbital twice)
degenerate_levels_list = degenerate_pair_gen(num_orbitals)      ## Degenerate level list, list of pair combinations that share the same energy
spin_pair_list = antipara_spin_pair_gen(obs_onebody_df,num_orbitals)  ## list of Pairs with T=1 (include Tz=0,+-1)
init_pair_list = HFground_pair_list(num_particles, num_orbitals)      ## Allowed Initial state
## to allow for set that include blocked levels for Be9, and set that cater for promotions in Be10

allowed_init = list(set(degenerate_levels_list) & set(spin_pair_list) & set(init_pair_list))
allowed_fina = list(set(degenerate_levels_list) & set(spin_pair_list))


### Allowed promotions
#### Neutron orbitals (indeces assigned to qubit)
neut_orbitals_list = list(range(0,num_orbitals[0]))
#### Proton orbitals (indeces assigned to qubit)
prot_orbitals_list = list(range(num_orbitals[0], sum(num_orbitals)))
excitations_list = []
for init in allowed_init:
    for fina in allowed_fina:
        if ((init[0] in neut_orbitals_list) == (fina[0] in neut_orbitals_list) and
            (init[1] in neut_orbitals_list) == (fina[1] in neut_orbitals_list) and
            (init[0] in prot_orbitals_list) == (fina[0] in prot_orbitals_list) and
            (init[1] in prot_orbitals_list) == (fina[1] in prot_orbitals_list)):
            excitations = [init,fina]; excitations.sort();
            excitations_list.append(tuple(excitations) if (excitations not in excitations_list) else tuple())
excitations_list.sort()

In [7]:
## create a drop_list that containes row index to drop
drop_list = []
for line_number, (index, row) in enumerate(obs_twobody_df.iterrows()):
    init_1 = int(row['q_i1']); init_2 = int(row['q_i2']);
    fina_1 = int(row['q_f1']); fina_2 = int(row['q_f2']);
    current_promotion = ((init_1,init_2),(fina_1,fina_2))
    if (current_promotion in excitations_list):
        pass
    else:
        drop_list.append(line_number)
obs_twobody_df.drop(drop_list,axis=0,inplace=True)
obs_twobody_df.reset_index(drop=True,inplace=True)

### To csv
obs_twobody_H_input_df_output = obs_twobody_df.copy()
obs_twobody_H_input_df_output.to_csv(pathfilename['output_2B-H_input_csv'])

# Create obs_onebody_H_input_df_output (input for VQE)

In [8]:
##### obs_onebody_df will always be the name of being_worked_df of onebody
del obs_onebody_df
obs_onebody_df = pd.DataFrame(columns =['q_i','q_f','delta'])

### init_fina_list
if_list = sin_bod_if_list_gen(num_orbitals)
epsilon_list = obs_onebody_source_df_output.loc[:,"epsilon"]

for counter, if_ in enumerate(if_list):
    delta = epsilon_list[if_[1]] - epsilon_list[if_[0]]
    obs_onebody_df.loc[counter] = [if_[0], if_[1], round(delta,6)]

obs_onebody_H_input_df_output = obs_onebody_df.copy()
obs_onebody_H_input_df_output.to_csv(pathfilename['output_1B-H_input_csv'])



In [9]:
%store excitations_list
%store num_orbitals
%store num_particles
%store num_spin_orbitals
%store obs_onebody_df
%store obs_twobody_df

Stored 'num_orbitals' (tuple)
Stored 'num_particles' (tuple)
Stored 'num_spin_orbitals' (int)
Stored 'obs_onebody_df' (DataFrame)
Stored 'obs_twobody_df' (DataFrame)


In [10]:
obs_onebody_df

,q_i,q_f,delta
0,0.0,1.0,0.000000
1,0.0,2.0,4.647110
2,0.0,3.0,4.647110
3,0.0,4.0,10.525158
4,0.0,5.0,10.525158
5,0.0,6.0,10.856729
6,0.0,7.0,10.856729
7,1.0,2.0,4.647110
8,1.0,3.0,4.647110
9,1.0,4.0,10.525158


[((0, 1), (0, 1)),
 ((0, 1), (2, 3)),
 ((0, 1), (4, 5)),
 ((0, 1), (6, 7)),
 ((0, 9), (0, 9)),
 ((0, 9), (1, 8)),
 ((0, 9), (1, 8)),
 ((0, 9), (2, 10)),
 ((0, 9), (3, 11)),
 ((0, 9), (4, 12)),
 ((0, 9), (5, 13)),
 ((0, 9), (6, 14)),
 ((0, 9), (7, 15)),
 ((1, 8), (1, 8)),
 ((1, 8), (2, 10)),
 ((1, 8), (3, 11)),
 ((1, 8), (4, 12)),
 ((1, 8), (5, 13)),
 ((1, 8), (6, 14)),
 ((1, 8), (7, 15)),
 ((8, 9), (8, 9)),
 ((8, 9), (10, 11)),
 ((8, 9), (12, 13)),
 ((8, 9), (14, 15))]